# Quantitative Momentum Strategy

"Momentum investing" means investing in the stocks that have increased in price the most.

For this project, we're going to build an investing strategy that selects the 50 stocks with the highest price momentum. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.


## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [113]:
!pip install xlsxwriter
!pip install python-dotenv

import os
import numpy as np
import pandas as pd
import requests
import math
from scipy.stats import percentileofscore as score
import xlsxwriter

from dotenv import load_dotenv
load_dotenv()

True

## Importing Our List of Stocks

As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the `.csv` file is still in your working directory and import it with the following command:

In [114]:
pwd = '/content/trading_practice/algorithmic-trading-python/starter_files'

stocks = pd.read_csv(f'{pwd}/sp_500_stocks.csv')
print(len(stocks.index))

token = os.getenv("IEX_CLOUD_API_TOKEN")
print(token)

505
Tpk_059b97af715d417d9f49f50b51b1c448


## Making Our First API Call

It's now time to make the first version of our momentum screener!

We need to get one-year price returns for each stock in the universe. Here's how.

In [115]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/stats?token={token}'
data = requests.get(api_url).json()
data

{'avg10Volume': 124160820,
 'avg30Volume': 108675361,
 'beta': 1.2061666640920932,
 'companyName': 'Apple Inc',
 'day200MovingAvg': 112.1,
 'day30ChangePercent': 0.14122233059080977,
 'day50MovingAvg': 123.78,
 'day5ChangePercent': 0.04326322296505171,
 'dividendYield': 0.006205917858346287,
 'employees': 143241,
 'exDividendDate': '2020-10-27',
 'float': 0,
 'marketcap': 2310466775393,
 'maxChangePercent': 50.2843132306365,
 'month1ChangePercent': 0.14226497640591382,
 'month3ChangePercent': 0.18575930813085864,
 'month6ChangePercent': 0.4585575766022686,
 'nextDividendDate': '0',
 'nextEarningsDate': '0',
 'peRatio': 40.134467658424875,
 'sharesOutstanding': 17158769195,
 'ttmDividendRate': 0.8249508634154723,
 'ttmEPS': 3.41,
 'week52change': 0.8668292580213418,
 'week52high': 137.78,
 'week52low': 58.06,
 'year1ChangePercent': 0.9062892767293468,
 'year2ChangePercent': 2.686355755278663,
 'year5ChangePercent': 4.484254657645296,
 'ytdChangePercent': 0.8498681883227499}

## Parsing Our API Call

This API call has all the information we need. We can parse it using the same square-bracket notation as in the first project of this course. Here is an example.

In [116]:
data['year1ChangePercent']

0.9062892767293468

## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called `chunks` that we can use to divide our list of securities into groups of 100.

In [117]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
    # print(symbol_strings[i])

my_columns = ['Ticker', 'Price', 'One-Year Price Return', 'Shares to Buy']

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [118]:
import json

df = pd.DataFrame(columns = my_columns)

batch_url = 'https://sandbox.iexapis.com/stable/stock/market/batch'

for symbol_string in symbol_strings:
    batch_api_call_url = f'{batch_url}/?types=stats,quote&symbols={symbol_string}&token={token}'
    data = requests.get(batch_api_call_url).json()

    for symbol in symbol_string.split(','):
        df = df.append(
            pd.Series(
                [
                    symbol, 
                    data[symbol]['quote']['latestPrice'],
                    data[symbol]['stats']['year1ChangePercent'],
                    'N/A'
                ], 
                index = my_columns), 
                ignore_index = True
                )
        
    
df

,Ticker,Price,One-Year Price Return,Shares to Buy
0,A,122.95,0.402539,N/A
1,AAL,16.30,-0.483274,N/A
2,AAP,166.61,0.00885897,N/A
3,AAPL,136.45,0.905654,N/A
4,ABBV,104.55,0.216894,N/A
...,...,...,...,...
500,YUM,108.87,0.0963982,N/A
501,ZBH,149.40,-0.00130184,N/A
502,ZBRA,386.94,0.505924,N/A
503,ZION,44.39,-0.129573,N/A


## Removing Low-Momentum Stocks

The investment strategy that we're building seeks to identify the 50 highest-momentum stocks in the S&P 500.

Because of this, the next thing we need to do is remove all the stocks in our DataFrame that fall below this momentum threshold. We'll sort the DataFrame by the stocks' one-year price return, and drop all stocks outside the top 50.


In [119]:
df.sort_values('One-Year Price Return', ascending = False, inplace = True)
df = df[:50]

df = df.reset_index(level=0, drop=True)

df.head()

,Ticker,Price,One-Year Price Return,Shares to Buy
0,CARR,39.53,2.2465,N/A
1,NVDA,528.17,1.20623,N/A
2,PYPL,246.34,1.20277,N/A
3,LB,40.12,1.17281,N/A
4,ALB,154.92,1.13109,N/A


## Calculating the Number of Shares to Buy

Just like in the last project, we now need to calculate the number of shares we need to buy. The one change we're going to make is wrapping this functionality inside a function, since we'll be using it again later in this Jupyter Notebook.

Since we've already done most of the work on this, try to complete the following two code cells without watching me do it first!

In [120]:
def portfolio_input():
    global p_size
    p_size = input('Enter the size of your portfolio: ')
    
    try:
        float(p_size)
    except ValueError:
        print("Not a number")
        portfolio_size()

portfolio_input()

In [121]:
position_size = float(p_size)/len(df)

for i in range(0, len(df)):
    df.loc[i, 'Shares to Buy'] = math.floor(position_size/df.loc[i, 'Price'])

df.head()

,Ticker,Price,One-Year Price Return,Shares to Buy
0,CARR,39.53,2.2465,5059
1,NVDA,528.17,1.20623,378
2,PYPL,246.34,1.20277,811
3,LB,40.12,1.17281,4985
4,ALB,154.92,1.13109,1290


## Building a Better (and More Realistic) Momentum Strategy

Real-world quantitative investment firms differentiate between "high quality" and "low quality" momentum stocks:

* High-quality momentum stocks show "slow and steady" outperformance over long periods of time
* Low-quality momentum stocks might not show any momentum for a long time, and then surge upwards.

The reason why high-quality momentum stocks are preferred is because low-quality momentum can often be cause by short-term news that is unlikely to be repeated in the future (such as an FDA approval for a biotechnology company).

To identify high-quality momentum, we're going to build a strategy that selects stocks from the highest percentiles of: 

* 1-month price returns
* 3-month price returns
* 6-month price returns
* 1-year price returns

Let's start by building our DataFrame. You'll notice that I use the abbreviation `hqm` often. It stands for `high-quality momentum`.

In [122]:
hqm_cols = [
    'Ticker',
    'Price',
    'Shares to Buy',
    '1Y Price Return',
    '1Y Return Percentile',
    '6M Price Return',
    '6M Return Percentile',
    '3M Price Return',
    '3M Return Percentile',
    '1M Price Return',
    '1M Return Percentile',
    'HQM Score'
]

hqm_df = pd.DataFrame(columns=hqm_cols)

for symbol_string in symbol_strings:
    batch_api_call_url = f'{batch_url}/?types=stats,quote&symbols={symbol_string}&token={token}'
    data = requests.get(batch_api_call_url).json()

    for symbol in symbol_string.split(','):
        hqm_df = hqm_df.append(
            pd.Series(
                [
                    symbol,
                    data[symbol]['quote']['latestPrice'],
                    'NA',
                    data[symbol]['stats']['year1ChangePercent'],
                    'NA',
                    data[symbol]['stats']['month6ChangePercent'],
                    'NA',
                    data[symbol]['stats']['month3ChangePercent'],
                    'NA',
                    data[symbol]['stats']['month1ChangePercent'],
                    'NA',
                    'NA'
                ], 
                index = hqm_cols), 
                ignore_index = True
                )
hqm_df

,Ticker,Price,Shares to Buy,1Y Price Return,1Y Return Percentile,6M Price Return,6M Return Percentile,3M Price Return,3M Return Percentile,1M Price Return,1M Return Percentile,HQM Score
0,A,123.11,NA,0.39187,NA,0.349718,NA,0.208783,NA,0.0262624,NA,NA
1,AAL,15.98,NA,-0.47898,NA,0.196907,NA,0.275425,NA,0.0495463,NA,NA
2,AAP,165.57,NA,0.00885747,NA,0.128471,NA,0.0788183,NA,0.0801797,NA,NA
3,AAPL,133.22,NA,0.880961,NA,0.45663,NA,0.178799,NA,0.137749,NA,NA
4,ABBV,103.60,NA,0.216597,NA,0.0928122,NA,0.218888,NA,-0.00916299,NA,NA
...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,108.58,NA,0.0965299,NA,0.273913,NA,0.182414,NA,0.0090911,NA,NA
501,ZBH,150.05,NA,-0.00130589,NA,0.303272,NA,0.111588,NA,-0.00130799,NA,NA
502,ZBRA,385.34,NA,0.485337,NA,0.502401,NA,0.475399,NA,0.0114366,NA,NA
503,ZION,43.93,NA,-0.133106,NA,0.297165,NA,0.548449,NA,0.060065,NA,NA


## Calculating Momentum Percentiles

We now need to calculate momentum percentile scores for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* `One-Year Price Return`
* `Six-Month Price Return`
* `Three-Month Price Return`
* `One-Month Price Return`

Here's how we'll do this:

In [123]:
time_periods = [
                '1Y',
                '6M',
                '3M',
                '1M'
                ]

for row in hqm_df.index:
    for time_period in time_periods:
        if hqm_df.loc[row, f'{time_period} Price Return'] == None:
            hqm_df.loc[row, f'{time_period} Price Return'] = 0

for row in hqm_df.index:
    for time_period in time_periods:

        change_col = f'{time_period} Price Return'
        percentile_col = f'{time_period} Return Percentile'

        hqm_df.loc[row, percentile_col] = score(
            hqm_df[change_col], 
            hqm_df.loc[row, change_col]
            )

# Print each percentile score to make sure it was calculated properly
# for time_period in time_periods:
#     print(hqm_df[f'{time_period} Return Percentile'])

#Print the entire DataFrame    
hqm_df

,Ticker,Price,Shares to Buy,1Y Price Return,1Y Return Percentile,6M Price Return,6M Return Percentile,3M Price Return,3M Return Percentile,1M Price Return,1M Return Percentile,HQM Score
0,A,123.11,NA,0.39187,84.7525,0.349718,68.3168,0.208783,57.2277,0.0262624,61.3861,NA
1,AAL,15.98,NA,-0.47898,1.78218,0.196907,41.5842,0.275425,71.2871,0.0495463,74.0594,NA
2,AAP,165.57,NA,0.00885747,40,0.128471,28.9109,0.0788183,29.703,0.0801797,85.1485,NA
3,AAPL,133.22,NA,0.880961,97.4257,0.45663,81.7822,0.178799,50.8911,0.137749,95.4455,NA
4,ABBV,103.60,NA,0.216597,68.7129,0.0928122,22.5743,0.218888,60.396,-0.00916299,35.2475,NA
...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,108.58,NA,0.0965299,52.8713,0.273913,56.4356,0.182414,51.4851,0.0090911,48.1188,NA
501,ZBH,150.05,NA,-0.00130589,37.4257,0.303272,61.9802,0.111588,37.8218,-0.00130799,39.802,NA
502,ZBRA,385.34,NA,0.485337,91.8812,0.502401,86.5347,0.475399,92.2772,0.0114366,50.297,NA
503,ZION,43.93,NA,-0.133106,21.1881,0.297165,60.396,0.548449,95.2475,0.060065,78.4158,NA


## Calculating the HQM Score

We'll now calculate our `HQM Score`, which is the high-quality momentum score that we'll use to filter for stocks in this investing strategy.

The `HQM Score` will be the arithmetic mean of the 4 momentum percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the `mean` function from Python's built-in `statistics` module.

In [124]:
from statistics import mean

for row in hqm_df.index:
    momentum_percentiles = []

    for time_period in time_periods:
        momentum_percentiles.append(
            hqm_df.loc[row, f'{time_period} Return Percentile']
        )
    
    hqm_df.loc[row, 'HQM Score'] = mean(momentum_percentiles)

hqm_df

,Ticker,Price,Shares to Buy,1Y Price Return,1Y Return Percentile,6M Price Return,6M Return Percentile,3M Price Return,3M Return Percentile,1M Price Return,1M Return Percentile,HQM Score
0,A,123.11,NA,0.39187,84.7525,0.349718,68.3168,0.208783,57.2277,0.0262624,61.3861,67.9208
1,AAL,15.98,NA,-0.47898,1.78218,0.196907,41.5842,0.275425,71.2871,0.0495463,74.0594,47.1782
2,AAP,165.57,NA,0.00885747,40,0.128471,28.9109,0.0788183,29.703,0.0801797,85.1485,45.9406
3,AAPL,133.22,NA,0.880961,97.4257,0.45663,81.7822,0.178799,50.8911,0.137749,95.4455,81.3861
4,ABBV,103.60,NA,0.216597,68.7129,0.0928122,22.5743,0.218888,60.396,-0.00916299,35.2475,46.7327
...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,108.58,NA,0.0965299,52.8713,0.273913,56.4356,0.182414,51.4851,0.0090911,48.1188,52.2277
501,ZBH,150.05,NA,-0.00130589,37.4257,0.303272,61.9802,0.111588,37.8218,-0.00130799,39.802,44.2574
502,ZBRA,385.34,NA,0.485337,91.8812,0.502401,86.5347,0.475399,92.2772,0.0114366,50.297,80.2475
503,ZION,43.93,NA,-0.133106,21.1881,0.297165,60.396,0.548449,95.2475,0.060065,78.4158,63.8119


## Selecting the 50 Best Momentum Stocks

As before, we can identify the 50 best momentum stocks in our universe by sorting the DataFrame on the `HQM Score` column and dropping all but the top 50 entries.

In [125]:
hqm_df.sort_values('HQM Score', ascending=False, inplace=True)

hqm_df = hqm_df[:50]
hqm_df.reset_index(level=0, inplace=True, drop=True)

hqm_df

,Ticker,Price,Shares to Buy,1Y Price Return,1Y Return Percentile,6M Price Return,6M Return Percentile,3M Price Return,3M Return Percentile,1M Price Return,1M Return Percentile,HQM Score
0,ALB,152.92,NA,1.13798,99.2079,1.05517,99.0099,0.779732,99.2079,0.119026,93.8614,97.8218
1,ALGN,528.62,NA,0.941433,98.6139,1.09092,99.4059,0.67935,98.6139,0.120058,94.2574,97.7228
2,PAYC,465.49,NA,0.779517,96.6337,0.476373,85.1485,0.633421,97.0297,0.1399,95.8416,93.6634
3,FCX,25.57,NA,0.932725,98.2178,1.29475,99.604,0.646991,97.2277,0.0531065,76.0396,92.7723
4,SIVB,390.36,NA,0.51241,92.2772,0.748922,96.2376,0.666753,98.2178,0.0613533,79.2079,91.4851
5,LRCX,501.32,NA,0.671611,95.0495,0.546959,90.297,0.48681,93.6634,0.0812466,85.7426,91.1881
6,IDXX,521.25,NA,0.898748,97.8218,0.542256,89.505,0.331301,79.802,0.117632,92.8713,90
7,APTV,130.78,NA,0.362535,82.5743,0.693903,94.8515,0.5024,94.2574,0.0853381,86.1386,89.4554
8,QRVO,170.77,NA,0.424843,87.9208,0.54952,90.6931,0.342301,81.7822,0.131249,94.8515,88.8119
9,TWTR,55.95,NA,0.680274,95.2475,0.727909,95.8416,0.23985,65.7426,0.170215,97.8218,88.6634


## Calculating the Number of Shares to Buy

We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a `for` loop to calculate the number of shares to buy for each stock in our investment universe.

In [126]:
portfolio_input()

In [127]:
position_size = float(p_size)/len(hqm_df.index)

for i in hqm_df.index:
    hqm_df.loc[i, 'Shares to Buy'] = math.floor(position_size/hqm_df.loc[i, 'Price'])

hqm_df

,Ticker,Price,Shares to Buy,1Y Price Return,1Y Return Percentile,6M Price Return,6M Return Percentile,3M Price Return,3M Return Percentile,1M Price Return,1M Return Percentile,HQM Score
0,ALB,152.92,130,1.13798,99.2079,1.05517,99.0099,0.779732,99.2079,0.119026,93.8614,97.8218
1,ALGN,528.62,37,0.941433,98.6139,1.09092,99.4059,0.67935,98.6139,0.120058,94.2574,97.7228
2,PAYC,465.49,42,0.779517,96.6337,0.476373,85.1485,0.633421,97.0297,0.1399,95.8416,93.6634
3,FCX,25.57,782,0.932725,98.2178,1.29475,99.604,0.646991,97.2277,0.0531065,76.0396,92.7723
4,SIVB,390.36,51,0.51241,92.2772,0.748922,96.2376,0.666753,98.2178,0.0613533,79.2079,91.4851
5,LRCX,501.32,39,0.671611,95.0495,0.546959,90.297,0.48681,93.6634,0.0812466,85.7426,91.1881
6,IDXX,521.25,38,0.898748,97.8218,0.542256,89.505,0.331301,79.802,0.117632,92.8713,90
7,APTV,130.78,152,0.362535,82.5743,0.693903,94.8515,0.5024,94.2574,0.0853381,86.1386,89.4554
8,QRVO,170.77,117,0.424843,87.9208,0.54952,90.6931,0.342301,81.7822,0.131249,94.8515,88.8119
9,TWTR,55.95,357,0.680274,95.2475,0.727909,95.8416,0.23985,65.7426,0.170215,97.8218,88.6634


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

## Creating the Formats We'll Need For Our .xlsx File

You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

Since we already built our formats in the last section of this course, I've included them below for you. Run this code cell before proceeding.

In [128]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

NameError: name 'writer' is not defined

## Saving Our Excel Output

As before, saving our Excel output is very easy: